In [1]:
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from PIL import Image
import tqdm
import os
import threading
import time

## Парсинг изображений человеков


In [2]:
def image_downloader(page, path):
    search = 'cyberpunk люди'
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/112.0'
    }
    cookies = {
        'cookie_name': 'cookie_value'
    }    

    url = f'https://ru.freepik.com/search?ai=exclude&format=search&page={page}&people=include&query=люди&type=photo'
    response = requests.get(url, headers = headers, cookies = cookies)
    html = response.content

    soup = BeautifulSoup(html, 'html.parser')
    figures = soup.find_all('figure', class_ = 'showcase__item js-detail-data caption showcase__item--buttons')

    for iteration, figure in enumerate(figures):
        url = figure.get('data-image')
        with Image.open(BytesIO(requests.get(url).content)) as im:
            im.save(f'{path}p{page}i{iteration}.jpg')
            
def make_threads(path, pages_amount = 200, threads_amount = 30):
    if not os.path.exists(path):
        os.makedirs(path)

    for pages in tqdm.tqdm(range(1, pages_amount+1, threads_amount)):
        threads = dict()
        for page in range(pages, pages+threads_amount+1):
            threads[page] = threading.Thread(target = image_downloader, args = [page, path])
            threads[page].start()
        for thread in threads.values():
            thread.join()

In [3]:
make_threads(path = 'images2/')

100%|█████████████████████████████████████████████| 7/7 [05:24<00:00, 46.35s/it]
